# Twitter Bot Detection
##### by Nefeli Tavoulari and Georgios Nikolaou
In this notebook, we are using a dataset from [Kaggle](https://www.kaggle.com/davidmartngutirrez/twitter-bots-accounts) and [Botometer](https://botometer.osome.iu.edu/bot-repository/datasets.html), containing in total 26K twitter user ids. Thanks to the Tweepy Python library we are retrieving users' information, which we considered useful in order to determine the type of an account.We have numerical, categorical and also textual features. So we train 3 models, one for the numerical/categorical features, another for the textual features and one more for the combination of the two types of features.
We also used different classifiers and different evaluation metrics, in order to make comparisons between the results.

### Install Dependencies

In [0]:
!sudo pip install --upgrade pip
!sudo pip install pyspark --upgrade
!pip install nltk
!python -m nltk.downloader all
!pip install stopwords
!pip install mlflow
!pip install langdetect
!pip install tweepy

Requirement already satisfied: pip in /usr/local/lib/python3.8/site-packages (21.1.1)
 WARNING: Running pip as root will break packages and permissions. You should install packages reliably by using venv: https://pip.pypa.io/warnings/venv 
Requirement already satisfied: pyspark in /usr/local/lib/python3.8/site-packages (3.1.1)
Requirement already satisfied: py4j==0.10.9 in /usr/local/lib/python3.8/site-packages (from pyspark) (0.10.9)
 WARNING: Running pip as root will break packages and permissions. You should install packages reliably by using venv: https://pip.pypa.io/warnings/venv 
Requirement already satisfied: nltk in /databricks/python3/lib/python3.8/site-packages (3.6.2)
Requirement already satisfied: regex in /databricks/python3/lib/python3.8/site-packages (from nltk) (2021.4.4)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from nltk) (8.0.1)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from nltk) (4.60.0)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (0.17.0)
 WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
/usr/local/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
 warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Package abc is already up-to-date!
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Package alpino is already up-to-date!
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package biocreative_ppi is already up-to-date!
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Package brown is already up-to-date!
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Package brown_tei is already up-to-date!
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Package cess_cat is already up-to-date!
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Package cess_esp is already up-to-date!
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Package chat80 is already up-to-date!
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package city_database is already up-to-date!
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Package cmudict is already up-to-date!
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package comparative_sentences is already up-to-
[nltk_data] | date!
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Package comtrans is already up-to-date!
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Package conll2000 is already up-to-date!
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Package conll2002 is already up-to-date!
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Package conll2007 is already up-to-date!
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Package crubadan is already up-to-date!
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package dependency_treebank is already up-to-date!
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Package dolch is already up-to-date!
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /ro

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import csv
import langdetect
import tweepy
from tweepy import OAuthHandler

import mlflow
import string
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import FMClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer, MinMaxScaler
from pyspark.sql.functions import udf, col

import datetime
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import re
from IPython.display import HTML

#### Some globals

In [0]:
now = datetime.datetime.now()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
RANDOM_SEED = 42 # For reproducibility
best_AUC=0 #For testing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [0]:
spark

Out[4]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

#### Import the Dataset

In [0]:
bot_df = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/shared_uploads/nikolaou.1977703@studenti.uniroma1.it/final__1_.csv")

#### Remove Duplicates

In [0]:
bot_df.dropDuplicates(["id"])

Out[6]: DataFrame[id: string, account_type: string, screen_name: string, follower_count: string, friends_count: string, listed_count: string, statuses_count: string, geo_enabled: string, verified: string, created_at: string, has_extended_profile: string, default_profile: string, default_profile_image: string, retweets: string, with_url: string, with_mention: string, avg_cosine: string, description: string, tweet_text: string]

#### Some UDFs

In [0]:
#Clean and perform stemming
def clean_text(text):
  porter = PorterStemmer()
  if text is None:
    return ""
  if text == []:
    return ""
  row = text.lower()
  row = row.strip() 
  row = re.sub(r'[^\w\s]',' ',row)
  row = re.sub(r'\_',' ',row)

  filtered_sentence = ""
  for w in row.split() :
    temp = porter.stem(w)
    filtered_sentence += (temp + " ")
  row = filtered_sentence

  if row is None:
    return ""
  if row == []:
    return ""
  return row
spark.udf.register("clean_text",clean_text)


# Get the days since the account was created
def to_days(then):
  now = datetime.datetime.now()
  date_time_obj = datetime.datetime.strptime(then, '%Y-%m-%d %H:%M:%S').date()
  diff = (now.date() - date_time_obj)
  diff = str(diff).split(' ')
  return int(diff[0])
to_days_UDF = spark.udf.register("to_daysUDF", to_days)

#### Split the dataframes into Numerical and Textual/Combined

In [0]:
# Get the days since account creation
bot_df = bot_df.withColumn("created_at", to_days_UDF(col("created_at")))

# The dataframe for both the Textual and Combined methods
bot_df_comb = bot_df.selectExpr("account_type","cast(follower_count as int) follower_count","cast(friends_count as int) friends_count","cast(listed_count as int) listed_count","cast(statuses_count as int) statuses_count","cast(retweets as float) retweets","cast(with_url as float) with_url","cast(with_mention as float) with_mention","geo_enabled", "verified", "has_extended_profile", "default_profile", "default_profile_image","cast(created_at as int) created_at","cast(avg_cosine as float) avg_cosine",'clean_text(description) as description','clean_text(tweet_text) as tweet_text')

# The dataframe for the Numerical Method
bot_df_num = bot_df.selectExpr("account_type","cast(follower_count as int) follower_count","cast(friends_count as int) friends_count","cast(listed_count as int) listed_count","cast(statuses_count as int) statuses_count","cast(retweets as float) retweets","cast(with_url as float) with_url","cast(with_mention as float) with_mention","geo_enabled", "verified", "has_extended_profile", "default_profile", "default_profile_image","cast(created_at as int) created_at","cast(avg_cosine as float) avg_cosine")

#### Split features into categories

In [0]:
NUMERICAL_FEATURES = ["follower_count", 
                      "friends_count",
                      "listed_count",
                      "statuses_count",
                      "retweets",
                      "with_url",
                      "with_mention",
                      "created_at",
                      "avg_cosine"
                      ]
CATEGORICAL_FEATURES = ["geo_enabled", 
                        "verified", 
                        "has_extended_profile",
                        "default_profile",
                        "default_profile_image",
                        ]

TEXTUAL_FEATURES =  ["description",
                     "tweet_text"
                     ]

TARGET_VARIABLE = "account_type"

#### Split in Train/Test

In [0]:
bot_df.groupBy(TARGET_VARIABLE).count().show()
train_df_both, test_df_both = bot_df_comb.randomSplit([0.8, 0.2], seed=RANDOM_SEED)
train_df_num, test_df_num = bot_df_num.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

+------------+-----+
account_type|count|
+------------+-----+
 human|11905|
 bot|13776|
+------------+-----+

#### Evaluate the Testing Results

In [0]:
def evaluate_model(predictions, metric="areaUnderROC"):    
  from pyspark.ml.evaluation import BinaryClassificationEvaluator
  evaluator = BinaryClassificationEvaluator(metricName=metric)
  return evaluator.evaluate(predictions)

#### Pipeline to create the train/test dataframes for both the combined and textual models

In [0]:
def pipeline_fitter(train, numerical_features, categorical_features, target_variable,with_std=True,with_mean=True):
    
    # Stage 1-3 to gets the word2vec for the tweet texts
    stage_1 = RegexTokenizer(inputCol="tweet_text", outputCol="tokens", pattern="\\W")
    
    stage_2 = StopWordsRemover(inputCol="tokens", outputCol="filtered_words")
    
    stage_3 = Word2Vec(inputCol="filtered_words", outputCol="feature_vector_text", vectorSize=1)
    
    # Stage 4-6 to gets the word2vec for the description
    stage_4 = RegexTokenizer(inputCol="description", outputCol="tokens_des", pattern="\\W")
    
    stage_5 = StopWordsRemover(inputCol="tokens_des", outputCol="filtered_words_des")
    
    stage_6 = Word2Vec(inputCol="filtered_words_des", outputCol="feature_vector_des", vectorSize=1)
    
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]
    
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")

    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Combine the features of both tweet texts, description and numericals, for the combined methods
    assembler_comb = VectorAssembler(inputCols=encoder.getOutputCols() + numerical_features+['feature_vector_des',"feature_vector_text"], outputCol="features_comb")
    
    # Combine the features of both tweet texts and description, for the textual methods
    assembler_text = VectorAssembler(inputCols=encoder.getOutputCols() +['feature_vector_des',"feature_vector_text"], outputCol="features_text")

    scaler_comb = StandardScaler(inputCol=assembler_comb.getOutputCol(), outputCol="std_"+assembler_comb.getOutputCol(), withStd=with_std, withMean=with_mean)
    
    scaler_text = StandardScaler(inputCol=assembler_text.getOutputCol(), outputCol="std_"+assembler_text.getOutputCol(), withStd=with_std, withMean=with_mean)
    # Combine the stages
    stages = indexers + [encoder] + [label_indexer]+[stage_1]+[stage_2]+[stage_3]+[stage_4]+[stage_5]+[stage_6]+[assembler_comb]+[assembler_text]+[scaler_comb]+[scaler_text]

    pipeline = Pipeline(stages=stages)
    
    transformer = pipeline.fit(train)
    df_transformed = transformer.transform(train)

    return transformer, df_transformed

In [0]:
transformer,train_df_both = pipeline_fitter(train_df_both, NUMERICAL_FEATURES, CATEGORICAL_FEATURES,  TARGET_VARIABLE)
test_df_both=transformer.transform(test_df_both)

# Get the train/ test set for both Combined and Textual methods
train_df_comb=train_df_both.select(["std_features_comb", "label"])
test_df_comb=test_df_both.select(["std_features_comb", "label"])

train_df_text=train_df_both.select(["std_features_text", "label"])
test_df_text=test_df_both.select(["std_features_text", "label"])

## Combined Textual and Numerical/Categorical Features

### 1. Logistic Regression

In [0]:
log_reg = LogisticRegression(featuresCol = "std_features_comb", labelCol = "label", maxIter=100)
log_reg_model = log_reg.fit(train_df_comb)

In [0]:
test_predictions = log_reg_model.transform(test_df_comb)
test_predictions.select("std_features_comb", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_comb|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))
print("***** Test Set *****")

best_model=log_reg_model
best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.892
Area Under Precision-Recall Curve: 0.879
***** Test Set *****

### 2. Random Forest

In [0]:
rf = RandomForestClassifier(featuresCol="std_features_comb", labelCol="label")
rf_model = rf.fit(train_df_comb)

In [0]:
test_predictions = rf_model.transform(test_df_comb)
test_predictions.select("std_features_comb", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_comb|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))
print("***** Test Set *****")

if evaluate_model(test_predictions)>best_AUC:
  best_model=rf_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.932
Area Under Precision-Recall Curve: 0.909
***** Test Set *****

### 3. Decision Tree

In [0]:
dt = DecisionTreeClassifier(featuresCol="std_features_comb", labelCol="label")
dt_model = dt.fit(train_df_comb)

In [0]:
test_predictions = dt_model.transform(test_df_comb)
test_predictions.select("std_features_comb", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_comb|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))
print("***** Test Set *****")

if evaluate_model(test_predictions)>best_AUC:
  best_model=dt_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.871
Area Under Precision-Recall Curve: 0.804
***** Test Set *****

### 4. Factorization machines classifier

In [0]:
fm = FMClassifier(labelCol="label", featuresCol="std_features_comb")
fm_model = fm.fit(train_df_comb)

In [0]:
test_predictions = fm_model.transform(test_df_comb)
test_predictions.select("std_features_comb", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_comb|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 1.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))
print("***** Test Set *****")

if evaluate_model(test_predictions)>best_AUC:
  best_model=fm_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.912
Area Under Precision-Recall Curve: 0.883
***** Test Set *****

## Numerical/Categorical Features

###1. Logistic Regression

In [0]:
def logistic_regression_pipeline(train, 
                                 numerical_features, 
                                 categorical_features, 
                                 target_variable, 
                                 with_std=True,
                                 with_mean=True,
                                 k_fold=5):

    # 1.a Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 1.b Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")

    # 1.c Indexing the target column (i.e., transform human/bot into 1/0) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # 1.d Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=encoder.getOutputCols() + numerical_features, outputCol="features")

    # 2.a Create the StandardScaler
    scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)
    # ...

    # 3 Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [encoder] + [label_indexer] + [assembler]  + [scaler] #+ ...

    # 4. Create the logistic regression transformer
    log_reg = LogisticRegression(featuresCol="std_features", labelCol="label", maxIter=100) # change `featuresCol=std_features` if scaler is used
    # 5. Add the logistic regression transformer to the pipeline stages (i.e., the last one)
    stages += [log_reg]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for log_reg.regParam ($\lambda$) and 3 values for log_reg.elasticNetParam ($\alpha$),
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(log_reg.regParam, [0.0, 0.05, 0.1]) \
    .addGrid(log_reg.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: lambda=[{:.3f}]; alpha=[{:.3f}] ".format(m.stages[-1]._java_obj.getRegParam(), m.stages[-1]._java_obj.getElasticNetParam()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***************************************\n")

In [0]:
cv_model = logistic_regression_pipeline(train_df_num, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)
summarize_all_models(cv_model.subModels)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
*************** Fold #1 ***************

--- Model #1 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #2 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #3 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #4 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #5 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #6 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #7 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #8 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #9 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

***************************************

*************** Fold #2 ***************

--- Model #1 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #2 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #3 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #4 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #5 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #6 out of 9 ---
	Parameters: lambda=[0.050]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #7 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #8 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #9 out of 9 ---
	Parameters: lambda=[0.100]; alpha=[1.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

***************************************

*************** Fold #3 ***************

--- Model #1 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.000] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #2 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[0.500] 
	Model summary: LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

--- Model #3 out of 9 ---
	Parameters: lambda=[0.000]; alpha=[1.000] 
	Model summary: LogisticRegressio

In [0]:
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    
print("Best model according to k-fold cross validation: lambda=[{:.3f}]; alfa=[{:.3f}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getRegParam(), 
             cv_model.bestModel.stages[-1]._java_obj.getElasticNetParam(),
             )
      )
print(cv_model.bestModel.stages[-1])

# `bestModel` is the best resulting model according to k-fold cross validation, which is also entirely retrained on the whole `train_df`
training_result = cv_model.bestModel.stages[-1].summary

Avg. ROC AUC computed across k-fold cross validation for model setting #1: 0.877
Avg. ROC AUC computed across k-fold cross validation for model setting #2: 0.877
Avg. ROC AUC computed across k-fold cross validation for model setting #3: 0.877
Avg. ROC AUC computed across k-fold cross validation for model setting #4: 0.869
Avg. ROC AUC computed across k-fold cross validation for model setting #5: 0.867
Avg. ROC AUC computed across k-fold cross validation for model setting #6: 0.861
Avg. ROC AUC computed across k-fold cross validation for model setting #7: 0.865
Avg. ROC AUC computed across k-fold cross validation for model setting #8: 0.863
Avg. ROC AUC computed across k-fold cross validation for model setting #9: 0.846
Best model according to k-fold cross validation: lambda=[0.000]; alfa=[1.000]
LogisticRegressionModel: uid=LogisticRegression_5f8644f4ef58, numClasses=2, numFeatures=24

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df_num)

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

if evaluate_model(test_predictions)>best_AUC:
  best_model=cv_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.891
Area Under Precision-Recall Curve: 0.878

#### Plots

In [0]:
plt.figure(figsize=(5,5))
# roc 
#plt.subplot(2, 6, 1)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.roc.select('FPR').collect(),
         training_result.roc.select('TPR').collect())
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

#plt.subplot(2, 6, 2)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.pr.select('recall').collect(),
         training_result.pr.select('precision').collect())
plt.xlabel('recall')
plt.ylabel('presicion')
plt.show()

#plt.subplot(2, 6, 3)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.precisionByThreshold.select('threshold').collect(),
         training_result.precisionByThreshold.select('precision').collect())
plt.xlabel('threshold')
plt.ylabel('precision')
plt.show()


#plt.subplot(2, 6, 4)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.recallByThreshold.select('threshold').collect(),
         training_result.recallByThreshold.select('recall').collect())
plt.xlabel('threshold')
plt.ylabel('recall')
plt.show()

#plt.subplot(2, 6, 5)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.fMeasureByThreshold.select('threshold').collect(),
         training_result.fMeasureByThreshold.select('F-Measure').collect())
plt.xlabel('threshold')
plt.ylabel('F-Measure')


plt.show()

###2. Desicion Tree

In [0]:
# This function defines the general pipeline for logistic regression
def decision_tree_pipeline(train, 
                           numerical_features, 
                           categorical_features, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):

    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # Indexing the target column (i.e., transform human/bot into 1/0) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    # Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [label_indexer] + [assembler] # + ...

    # Create the decision tree transformer
    dt = DecisionTreeClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [dt]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for dt.maxDepth and 2 values for dt.impurity
    # this grid will have 3 x 2 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [3, 5, 8]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: maxDept=[{:d}]; impurity=[{:s}] ".format(m.stages[-1]._java_obj.getMaxDepth(), m.stages[-1]._java_obj.getImpurity()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***************************************\n")

In [0]:
cv_model = decision_tree_pipeline(train_df_num, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)
summarize_all_models(cv_model.subModels)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
*************** Fold #1 ***************

--- Model #1 out of 6 ---
	Parameters: maxDept=[3]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=7, numClasses=2, numFeatures=14

--- Model #2 out of 6 ---
	Parameters: maxDept=[3]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=7, numClasses=2, numFeatures=14

--- Model #3 out of 6 ---
	Parameters: maxDept=[5]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=33, numClasses=2, numFeatures=14

--- Model #4 out of 6 ---
	Parameters: maxDept=[5]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=29, numClasses=2, numFeatures=14

--- Model #5 out of 6 ---
	Parameters: maxDept=[8]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=289, numClasses=2, numFeatures=14

--- Model #6 out of 6 ---
	Parameters: maxDept=[8]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=249, numClasses=2, numFeatures=14

***************************************

*************** Fold #2 ***************

--- Model #1 out of 6 ---
	Parameters: maxDept=[3]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=9, numClasses=2, numFeatures=14

--- Model #2 out of 6 ---
	Parameters: maxDept=[3]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=9, numClasses=2, numFeatures=14

--- Model #3 out of 6 ---
	Parameters: maxDept=[5]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=43, numClasses=2, numFeatures=14

--- Model #4 out of 6 ---
	Parameters: maxDept=[5]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=31, numClasses=2, numFeatures=14

--- Model #5 out of 6 ---
	Parameters: maxDept=[8]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=299, numClasses=2, numFeatures=14

--- Model #6 out of 6 ---
	Parameters: maxDept=[8]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=277, numClasses=2, numFeatures=14

***************************************

*************** Fold #3 ***************

--- Model #1 out of 6 ---
	Parameters: maxDept=[3]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=7, numClasses=2, numFeatures=14

--- Model #2 out of 6 ---
	Parameters: maxDept=[3]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=3, numNodes=7, numClasses=2, numFeatures=14

--- Model #3 out of 6 ---
	Parameters: maxDept=[5]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=41, numClasses=2, numFeatures=14

--- Model #4 out of 6 ---
	Parameters: maxDept=[5]; impurity=[entropy] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=5, numNodes=31, numClasses=2, numFeatures=14

--- Model #5 out of 6 ---
	Parameters: maxDept=[8]; impurity=[gini] 
	Model summary: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=321, numClasses=2, numFeatures=14

--- Model #6 out of 6 ---
	Parameters: maxDept=[8]; i

In [0]:
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if training_result < avg_roc_auc:
      training_result = avg_roc_auc
      
print("Best model according to k-fold cross validation: maxDept=[{:d}]; impurity=[{:s}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             cv_model.bestModel.stages[-1]._java_obj.getImpurity(),
             )
      )
print(cv_model.bestModel.stages[-1])

Avg. ROC AUC computed across k-fold cross validation for model setting #1: 0.822
Avg. ROC AUC computed across k-fold cross validation for model setting #2: 0.828
Avg. ROC AUC computed across k-fold cross validation for model setting #3: 0.870
Avg. ROC AUC computed across k-fold cross validation for model setting #4: 0.858
Avg. ROC AUC computed across k-fold cross validation for model setting #5: 0.870
Avg. ROC AUC computed across k-fold cross validation for model setting #6: 0.881
Best model according to k-fold cross validation: maxDept=[8]; impurity=[entropy]
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_133e729b22cd, depth=8, numNodes=261, numClasses=2, numFeatures=14

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df_num)
test_predictions.select("features", "prediction", "label").show(5)

+--------------------+----------+-----+
 features|prediction|label|
+--------------------+----------+-----+
(14,[3,4,8,9,10,1...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

if evaluate_model(test_predictions)>best_AUC:
  best_model=cv_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.898
Area Under Precision-Recall Curve: 0.840

###3. Random Forests

In [0]:
# This function defines the general pipeline for logistic regression
def random_forest_pipeline(train, 
                           numerical_features, 
                           categorical_features, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):


    # Configure a random forest pipeline, which consists of the following stages: 

    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # Indexing the target column (i.e., transform human/bot into 1/0) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    # Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [label_indexer] + [assembler] # + ...

    # Create the random forest transformer
    rf = RandomForestClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the random forest transformer to the pipeline stages (i.e., the last one)
    stages += [rf]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for rf.maxDepth and 3 values for rf.numTrees
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [3, 5, 8]) \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .build()
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model


In [0]:
cv_model = random_forest_pipeline(train_df_num, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if avg_roc_auc > training_result:
      training_result = avg_roc_auc
      
print("Best model according to k-fold cross validation: maxDept=[{:d}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(),),)

print(cv_model.bestModel.stages[-1])

Avg. ROC AUC computed across k-fold cross validation for model setting #1: 0.908
Avg. ROC AUC computed across k-fold cross validation for model setting #2: 0.914
Avg. ROC AUC computed across k-fold cross validation for model setting #3: 0.915
Avg. ROC AUC computed across k-fold cross validation for model setting #4: 0.927
Avg. ROC AUC computed across k-fold cross validation for model setting #5: 0.930
Avg. ROC AUC computed across k-fold cross validation for model setting #6: 0.932
Avg. ROC AUC computed across k-fold cross validation for model setting #7: 0.943
Avg. ROC AUC computed across k-fold cross validation for model setting #8: 0.946
Avg. ROC AUC computed across k-fold cross validation for model setting #9: 0.947
Best model according to k-fold cross validation: maxDept=[8]
RandomForestClassificationModel: uid=RandomForestClassifier_9af106b68dae, numTrees=100, numClasses=2, numFeatures=14

In [0]:
#training_result = cv_model.bestModel.stages[-1].summary
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df_num)
test_predictions.select("features", "prediction", "label").show(5)

+--------------------+----------+-----+
 features|prediction|label|
+--------------------+----------+-----+
(14,[3,4,8,9,10,1...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

if evaluate_model(test_predictions)>best_AUC:
  best_model=cv_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.948
Area Under Precision-Recall Curve: 0.934

###4. Factorization Machines

In [0]:
def fm_pipeline(train, 
                numerical_features,
                categorical_features, 
                target_variable, 
                with_std=True,
                with_mean=True,
                k_fold=5):



    # 1.a Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 1.b Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")


    # Indexing the target column (i.e., transform human/bot into 1/0) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
    
    stages = indexers + [encoder] + [label_indexer] + [assembler] + [featureScaler]
    
    fm = FMClassifier(labelCol="label", featuresCol="scaledFeatures")

    # 5. Add the decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [fm]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(fm.stepSize, [0.001,0.002,0.005,0.01]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = fm_pipeline(train_df_num, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if avg_roc_auc > training_result:
      training_result = avg_roc_auc

Avg. ROC AUC computed across k-fold cross validation for model setting #1: 0.838
Avg. ROC AUC computed across k-fold cross validation for model setting #2: 0.854
Avg. ROC AUC computed across k-fold cross validation for model setting #3: 0.875
Avg. ROC AUC computed across k-fold cross validation for model setting #4: 0.881

In [0]:
#training_result = cv_model.bestModel.stages[-1].summary
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions.select("features", "prediction", "label").show(5)

+--------------------+----------+-----+
 features|prediction|label|
+--------------------+----------+-----+
(14,[3,4,8,9,10,1...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
(14,[3,8,9,10,11,...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))


if evaluate_model(test_predictions)>best_AUC:
  best_model=cv_model
  best_AUC=evaluate_model(test_predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.948
Area Under Precision-Recall Curve: 0.934

## Textual Features

###1. Logistic Regression

In [0]:
log_reg = LogisticRegression(featuresCol = "std_features_text", labelCol = "label", maxIter=100)
log_reg_model = log_reg.fit(train_df_text)

In [0]:
predictions = log_reg_model.transform(test_df_text)
predictions.select("std_features_text", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_text|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(predictions, metric="areaUnderPR")))

if evaluate_model(predictions)>best_AUC:
  best_model=log_reg_model
  best_AUC=evaluate_model(predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.837
Area Under Precision-Recall Curve: 0.833

###2. Decision Tree

In [0]:
dt = DecisionTreeClassifier(featuresCol="std_features_text", labelCol="label")
dt_model = dt.fit(train_df_text)

In [0]:
predictions = dt_model.transform(test_df_text)
predictions.select("std_features_text", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_text|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(predictions, metric="areaUnderPR")))

if evaluate_model(predictions)>best_AUC:
  best_model=dt_model
  best_AUC=evaluate_model(predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.769
Area Under Precision-Recall Curve: 0.725

###3. Random Forests

In [0]:
rf = RandomForestClassifier(featuresCol="std_features_text", labelCol="label")
rf_model = rf.fit(train_df_text)

In [0]:
predictions = rf_model.transform(test_df_text)
predictions.select("std_features_text", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_text|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(predictions, metric="areaUnderPR")))


if evaluate_model(predictions)>best_AUC:
  best_model=rf_model
  best_AUC=evaluate_model(predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.881
Area Under Precision-Recall Curve: 0.869

###4. Factorization Machines

In [0]:
fm = FMClassifier(labelCol="label", featuresCol="std_features_text")
fm_model = fm.fit(train_df_text)

In [0]:
predictions = fm_model.transform(test_df_text)
predictions.select("std_features_text", "prediction", "label").show(5)

+--------------------+----------+-----+
 std_features_text|prediction|label|
+--------------------+----------+-----+
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
[0.78023779463755...| 0.0| 0.0|
+--------------------+----------+-----+
only showing top 5 rows

#### Evaluation

In [0]:
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(predictions, metric="areaUnderPR")))


if evaluate_model(predictions)>best_AUC:
  best_model=fm_model
  best_AUC=evaluate_model(predictions)

***** Test Set *****
Area Under ROC Curve (ROC AUC): 0.834
Area Under Precision-Recall Curve: 0.833

## Test the Results
Give a Twitter user's screen name on the widget above and find out if it is a bot or a human

In [0]:
def find_user(usr,model):
    text_list = []
    
    consumer_key = "phCKVDVUS7nBmCvN5aJZWwrxo"
    consumer_secret = "3k7gMiVmxPPDI0C6kTc8uMTL0nSdNfeeU82OGcNVftkaMmujlR"
    access_token = "1389540894022545408-lovri9oSZKdLryO5JXqvfwuLeruEGq"
    access_token_secret = "N0XU2cNl14QsO13IRDHCSFmuZFpELKqNG2pA9mkwaQfrg"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    try:
        # count
        retweets = 0
        with_mention = 0
        with_url = 0
        text = ""

        user = api.get_user(usr)
        tweets = api.user_timeline(
            screen_name=user.screen_name, count=130, include_rts=True, tweet_mode='extended')

        # read 130 of user's tweets
        for tweet in tweets:
            try:
                if tweet.retweeted_status:
                    retweets += 1
            except AttributeError:
                # combine all tweets into one big text
                to_add = remove_emoji(tweet.full_text).replace("\n", " ")
                to_add = clean(to_add)
                try:
                    # keep only english texts
                    if langdetect.detect(to_add) != 'en':
                        continue
                except langdetect.lang_detect_exception.LangDetectException:
                    continue
                text_list.append(to_add)
                text = text + " " + to_add
            if tweet.entities['urls']:
                with_url += 1
            if tweet.entities['user_mentions']:
                with_mention += 1

        text = " ".join(text.split())
        # find retweets,mentions and urls per tweet
        if len(tweets) >= 1:
            retweets = retweets / len(tweets)
            with_mention = with_mention / len(tweets)
            with_url = with_url/len(tweets)
        else:
            retweets = 0
            with_url = 0
            with_mention = 0

        # clean description
        description = " ".join((re.sub(r"(?:\@|http?\://|https?\://|www)\S+",
                                       "", remove_emoji(user.description).replace("\n", " "))).split())
        # create a new clean and complete csv file with the dataset

        bot_df = spark.createDataFrame(
            [
                ("human", user.followers_count, user.friends_count, user.listed_count, user.statuses_count, str(user.geo_enabled), str(user.verified),
                 str(user.created_at), str(user.has_extended_profile), str(user.default_profile), str(user.default_profile_image), retweets, with_url, with_mention, description, text),
            ],
            ['account_type', 'follower_count', 'friends_count', 'listed_count', 'statuses_count', 'geo_enabled', 'verified',
             'created_at', 'has_extended_profile', 'default_profile', 'default_profile_image', 'retweets', 'with_url', 'with_mention', 'description', 'tweet_text']  # add your column names here
        )

        bot_df = bot_df.withColumn(
            "created_at", to_days_UDF(col("created_at")))

        bot_df = bot_df.selectExpr("account_type", "cast(follower_count as int) follower_count", "cast(friends_count as int) friends_count", "cast(listed_count as int) listed_count", "cast(statuses_count as int) statuses_count",
                                   "cast(retweets as float) retweets", "cast(with_url as float) with_url", "cast(with_mention as float) with_mention", "geo_enabled", "verified", "has_extended_profile", "default_profile", "default_profile_image", "cast(created_at as int) created_at")

        test_predictions = model.transform(bot_df)

        # test_predictions.select("features", "prediction").show(1)
        test_predictions = test_predictions.select("prediction")

        bot_pdf = test_predictions.toPandas()

        
        if bot_pdf["prediction"][0] == 0:
            return "bot"
        else:
            return "human"

    except tweepy.TweepError:
      return "unknown"


In [0]:
dbutils.widgets.text("name", "")
find_user(dbutils.widgets.get("name"),best_model)

Out[37]: 'unknown'

## A preview of our Website's frontend

In [0]:
s = """<!DOCTYPE html>
<html>
        <head>
                <title>Bot Detector Website</title>
        <link rel= "stylesheet" type= "text/css" href= "{{
url_for('static',filename='styles/index.css') }}">
        </head>
        <body>
        <div class="container">
            <h1>Twitter Bot Detector</h1> 
            <p>Check if a user is human or bot.</p>
            <form action="" method="post">
                <input type="text" placeholder="e.g. @hello_kitty123" name="name">
                <div class="bar"></div>
                <div class="highlight"></div>
                <button class="btn striped-shadow dark" type="submit"
value="submit"><span>Check</span></button>
            </form>

        </div>
        </body>
    
    <style>
body {
  font-family: courier, arial, helvetica;
  font-size: 150%;
  color: #77bfa1;
  background-color: #726da8;
}

.container {
  text-align: center;
  margin-top: 10%;
  font-weight: bold;
}

@import "https://fonts.googleapis.com/css?family=Bungee+Shade";

*,
:after,
:before {
  box-sizing: border-box;
}

:focus {
  outline: none;
}

button {
  overflow: visible;
  border: 0;
  padding: 0;
  margin: 1.8rem;
}
.btn.striped-shadow span {
  display: block;
  position: relative;
  z-index: 2;
  border: 5px solid;
}

.btn.striped-shadow.dark span {
  border-color: #393939;
  background: #77bfa1;
  color: #393939;
}

.btn {
  height: 80px;
  line-height: 65px;
  display: inline-block;
  letter-spacing: 1px;
  position: relative;
  font-size: 1.35rem;
  transition: opacity 0.3s, z-index 0.3s step-end, -webkit-transform 0.3s;
  transition: opacity 0.3s, z-index 0.3s step-end, transform 0.3s;
  transition: opacity 0.3s, z-index 0.3s step-end, transform 0.3s,
    -webkit-transform 0.3s;
  z-index: 1;
  background-color: transparent;
  cursor: pointer;
}

.btn {
  width: 155px;
  height: 48px;
  line-height: 38px;
}

button.btn.striped-shadow.dark:after,
button.btn.striped-shadow.dark:before {
  background-image: linear-gradient(
    135deg,
    transparent 0,
    transparent 5px,
    #393939 5px,
    #393939 10px,
    transparent 10px
  );
}

button.btn.striped-shadow:hover:before {
  max-height: calc(100% - 10px);
}

button.btn.striped-shadow:after {
  width: calc(100% - 4px);
  height: 8px;
  left: -10px;
  bottom: -9px;
  background-size: 15px 8px;
  background-repeat: repeat-x;
}
button.btn.striped-shadow:after,
button.btn.striped-shadow:before {
  content: "";
  display: block;
  position: absolute;
  z-index: 1;
  transition: max-height 0.3s, width 0.3s, -webkit-transform 0.3s;
  transition: transform 0.3s, max-height 0.3s, width 0.3s;
  transition: transform 0.3s, max-height 0.3s, width 0.3s,
    -webkit-transform 0.3s;
}

.btn.striped-shadow:hover {
  -webkit-transform: translate(-12px, 12px);
  -ms-transform: translate(-12px, 12px);
  transform: translate(-12px, 12px);
  z-index: 3;
}

button.btn.striped-shadow:hover:after,
button.btn.striped-shadow:hover:before {
  -webkit-transform: translate(12px, -12px);
  -ms-transform: translate(12px, -12px);
  transform: translate(12px, -12px);
}
button.btn.striped-shadow:before {
  width: 8px;
  max-height: calc(100% - 5px);
  height: 100%;
  left: -12px;
  bottom: -5px;
  background-size: 8px 15px;
  background-repeat: repeat-y;
  background-position: 0 100%;
}

.input {
  margin: 5% 10%;
  position: relative;
  width: fit-content;
}
input {
  padding: 10px 10px 10px 5px;
  font-size: 18px;
  width: 280px;
  border: 1px solid;
  border-color: transparent transparent gray;
  background-color: transparent;
}
input:focus {
  outline: none;
}
/*Label */
label {
  position: absolute;
  top: 30%;
  font-size: 18px;
  color: rgb(165, 165, 165);
  left: 3%;
  z-index: -1;
  pointer-events: none;
  transition: all 0.3s;
  -webkit-transition: all 0.3s;
  -moz-transition: all 0.3s;
  -ms-transition: all 0.3s;
  -o-transition: all 0.3s;
}
/* Activate State */
input:focus + label,
input:valid + label {
  font-size: 12px;
  color: rgb(148, 98, 255);
  top: -1%;
  transition: all 0.3s;
  -webkit-transition: all 0.3s;
  -moz-transition: all 0.3s;
  -ms-transition: all 0.3s;
  -o-transition: all 0.3s;
}
/*End Label */
/*Bar*/
.bar {
  width: 100%;
  height: 2px;
  position: absolute;
  background-color: rgb(148, 98, 255);
  top: calc(100% - 2px);
  left: 0;
  transform: scaleX(0);
  -webkit-transform: scaleX(0);
  -moz-transform: scaleX(0);
  -ms-transform: scaleX(0);
  -o-transform: scaleX(0);
}
/*Activate State */
input:focus ~ .bar,
input:valid ~ .bar {
  transform: scaleX(1);
  -webkit-transform: scaleX(1);
  -moz-transform: scaleX(1);
  -ms-transform: scaleX(1);
  -o-transform: scaleX(1);
  transition: transform 0.3s;
  -webkit-transition: transform 0.3s;
  -moz-transition: transform 0.3s;
  -ms-transition: transform 0.3s;
  -o-transition: transform 0.3s;
}
/*End Bar */
/*Highlight */
.highlight {
  width: 100%;
  height: 85%;
  position: absolute;
  background-color: rgba(148, 98, 255, 0.2);
  top: 15%;
  left: 0;
  visibility: hidden;
  z-index: -1;
}
input:focus ~ .highlight {
  width: 0;
  visibility: visible;
  transition: all 0.09s linear;
  -webkit-transition: all 0.09s linear;
  -moz-transition: all 0.09s linear;
  -ms-transition: all 0.09s linear;
  -o-transition: all 0.09s linear;
}
/*End highlight */
::placeholder {
  color: #77bfa1;
  font-size: 20px;
}
</style>

</html>"""
h = HTML(s)
display(h)

<!DOCTYPE html>
 
 
 Bot Detector Website 
 
 
 
 
 Twitter Bot Detector 
 Check if a user is human or bot. 
 
 
 
 
 Check